# Extra Modelling Challanges

In this session we will work on symmetry and dominance breaking

In [1]:
# Setup
!pip install --upgrade cpmpy --quiet
! pip install matplotlib --quiet
import cpmpy as cp
import numpy as np
import matplotlib.pyplot as plt

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 152.0/152.0 kB 5.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 28.1/28.1 MB 61.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 133.7/133.7 kB 11.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.8/302.8 kB 23.9 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tensorflow 2.17.1 requires protobuf!=4.21.0,!=4.21.1,!=4.21.2,!=4.21.3,!=4.21.4,!=4.21.5,<5.0.0dev,>=3.20.3, but you have protobuf 5.26.1 which is incompatible.
tensorflow-metadata 1.13.1 requires absl-py<2.0.0,>=0.9, but you have absl-py 2.1.0 which is incompatible.
tensorflow-metadata 1.13.1 requires protobuf<5,>=3.20.3, but you have protobuf 5.26.1 which is incompatible.


**Cabling problem**
Let's say, there are 8 devices, maybe servers, routers etc, named as A, B, C, D, E, F, G, H.
Devices must be placed in a sequence, and consecutive devices must be connected by
cables. Some devices must be connected by several cables (2 cables, 3 or 4):
A <--- 1 cable ---> H
A <--- 2 cables ---> E
B <--- 4 cables ---> F
C <--- 1 cable ---> G
C <--- 1 cable ---> D
C <--- 1 cable ---> E
D <--- 3 cables ---> H
G <--- 1 cable ---> H
How we can place these 8 devices in such an order, so that sum of cables used would be as
short as possible?

In [ ]:
# Devices
devices = ["A","B","C","D","E","F","G","H"]

# Weights: number of cables between pairs
# We store them in a dictionary for convenience:
cables = {
    ("A","H"): 1,
    ("A","E"): 2,
    ("B","F"): 4,
    ("C","G"): 1,
    ("C","D"): 1,
    ("C","E"): 1,
    ("D","H"): 3,
    ("G","H"): 1
}

# Create position variables: pos[X] in 0..7
pos_var = { X: cp.intvar(0,7) for X in devices }

model = cp.Model()

# All different constraint
model += cp.AllDifferent(pos_var.values())

# Objective: sum of cable-distances

total_cost=0
for (x,y),w in cables.items():
  total_cost += w*(abs(pos_var[x]-pos_var[y]))

model.minimize(total_cost)

# Solve
if model.solve():
    print("Solution found:")
    for device, position in pos_var.items():
        print(f"{device}: {position.value()}")
    print(f"Total Cost: {total_cost.value()}")
else:
    print("No solution found")

Solution found:
A: 7
B: 0
C: 3
D: 4
E: 6
F: 1
G: 2
H: 5
Total Cost: 19


**Progressive Party Problem (PPP)**
The problem is to timetable a party at a yacht club. Certain boats are to be designated hosts,
and the crews of the remaining boats in turn visit the host boats for several successive halfhour
periods. The crew of a host boat remains on board to act as hosts while the crew of a
guest boat together visits several hosts. Every boat can only hold a limited number of people
at a time (its capacity) and crew sizes are different. The total number of people aboard a
boat, including the host crew and guest crews, must not exceed the capacity. A guest boat
cannot not revisit a host and guest crews cannot meet more than once.
The problem the rally organizer is facing is that of minimizing the number of host boats. Find
the visit schedule and the host designations.

In [106]:
schedule[4, :, :]

array([[0, 0, 0, 0],
       [0, 0, 0, 0],
       [0, 0, 0, 0],
       [0, 0, 0, 0],
       [0, 0, 0, 0],
       [0, 0, 0, 0],
       [0, 0, 0, 0],
       [0, 0, 0, 0],
       [0, 0, 0, 0],
       [0, 0, 0, 0]])

In [107]:
#to be went through

start_time = 10 # the party starts at 10am
end_time = 12 # the party ends at 12am
crew_size = cp.cpm_array([1, 2, 3, 4, 5, 6, 7, 8, 9, 10]) # number of people per crew
boat_capacity = cp.cpm_array([10, 10, 15, 20, 5, 8, 15, 10, 10, 12]) # maximum number of people per boat

T = 2 * (end_time - start_time) # each period lasts for 30min so there are 2 visits/hour
N = len(boat_capacity) # total number of crews = total number of boat

host = cp.boolvar(shape=N) # whether boat i is a host boat
schedule = cp.boolvar(shape=(N, N, T)) # i, j, k represents whether boat i carry crew j at time k. i is a boat, j is a crew

model = cp.Model()

model += [boat.implies(cp.all(schedule[i, i, t] == 1 for t in range(T))) for i, boat in enumerate(host)] # if a boat is a host boat, then its crew stays on their boat for the full party # for boat 1 crew 1 is always there for every time
model += [boat.implies(cp.sum(schedule[i, :, t] * crew_size) <= boat_capacity[i]) for t in range(T) for i, boat in enumerate(host)] # if a boat is a host boat, then the number of people on it per time period cannot exceed the capacity
model += [cp.sum(schedule[:, n, t]) == 1 for t in range(T) for n in range(N)] # each crew should be in exactly one host boat per time period
model += [(cp.sum(schedule[n, :, :]) > 0).implies(host[n]) for n in range(N)] # if a boat is used at some moment in time by some crew, then it must be a host boat

for j in range(N):
  for k in range(j):
    model += [cp.sum(schedule[:, j, :] * schedule[:, k, :]) <= 1] # the crews j and k can meet at most once one the same boat at the same time

model.minimize(cp.sum(host)) # minimise the total number of host boats

if model.solve():
  host_boats = np.where(host.value().astype(int) == 1)[0]
  schedule = schedule.value().astype(int)

  print("The host boats are:")
  for boat in host_boats:
    print("  - Boat", boat, "(maximum capacity", boat_capacity[boat], ") carries:")
    crews = schedule[boat, :, :]
    results = {}
    for i, time in enumerate(crews.T):
      crew = np.where(time == 1)[0]
      if len(crew) > 0:
        print("    at ", f"{int(10 + 0.5*i)}:{int(((10 + 0.5*i) * 60) % 60):02d}", ": crew(s)", ", ".join([str(c) for c in crew]), "->", "+".join([str(crew_size[c]) for c in crew]), "=", sum([crew_size[c] for c in crew]), "people")
else:
  print("No solution found.")

The host boats are:
  - Boat 1 (maximum capacity 10 ) carries:
    at  10:00 : crew(s) 1 -> 2 = 2 people
    at  10:30 : crew(s) 1, 5 -> 2+6 = 8 people
    at  11:00 : crew(s) 1, 7 -> 2+8 = 10 people
    at  11:30 : crew(s) 1, 4 -> 2+5 = 7 people
  - Boat 2 (maximum capacity 15 ) carries:
    at  10:00 : crew(s) 2, 7 -> 3+8 = 11 people
    at  10:30 : crew(s) 2, 4 -> 3+5 = 8 people
    at  11:00 : crew(s) 2, 5 -> 3+6 = 9 people
    at  11:30 : crew(s) 0, 2 -> 1+3 = 4 people
  - Boat 3 (maximum capacity 20 ) carries:
    at  10:00 : crew(s) 3, 4 -> 4+5 = 9 people
    at  10:30 : crew(s) 3, 7 -> 4+8 = 12 people
    at  11:00 : crew(s) 3 -> 4 = 4 people
    at  11:30 : crew(s) 3, 5 -> 4+6 = 10 people
  - Boat 6 (maximum capacity 15 ) carries:
    at  10:00 : crew(s) 5, 6 -> 6+7 = 13 people
    at  10:30 : crew(s) 0, 6 -> 1+7 = 8 people
    at  11:00 : crew(s) 4, 6 -> 5+7 = 12 people
    at  11:30 : crew(s) 6, 7 -> 7+8 = 15 people
  - Boat 8 (maximum capacity 10 ) carries:
    at  10:00 : 

**Devil's word**
Given an array of integers, insert '+' or '-' operators before each integer in the array so that
summing them up will result in 666. Given any array, find the correct signs to be used.

In [ ]:
# An example array of integers. Replace with your own data:
A = [666, 1, 1]

n = len(A)
take = cp.boolvar(shape=n)
model  =cp.Model()

model += sum((1-2*take[i])*A[i] for i in range(n))==666
if model.solve():
  print("Solution found")
  print("x:", take.value())
else:
  print("No solution found")

Solution found
x: [False False  True]


**Warehouse Location problem**
In the Warehouse Location problem, a company considers opening warehouses at some
candidate locations in order to supply its existing stores. Each possible warehouse has the
same maintenance cost, and a capacity designating the maximum number of stores that it
can supply. Each store must be supplied by exactly one open warehouse. The supply cost to
a store depends is different for each warehouse.
A company is considering opening warehouses in four cities to meet regional demands at
minimal costs. The potential cities for these warehouses are New York, Los Angeles,
Chicago, and Atlanta. The company needs to decide which warehouses to open based on
several constraints: 1. If the New York warehouse is opened, then the Los Angeles
warehouse must also be opened. 2. No more than three warehouses can be operational. 3.
Either the Atlanta or the Los Angeles warehouse must be opened.
The objective is to determine the optimal set of warehouses to open, and which of these
warehouses should supply the various stores, such that supply costs is minimized. Supply
costs for each warehouse-store combination should be given.

In [ ]:
#NY:30, LA:50, CH:20, A:10
Costs = [30, 50, 20, 10]
take = cp.boolvar(shape=4)

model=cp.Model()
model+= (take[0]==1).implies(take[1]==1)
model+= sum(take)<=3
model+= (take[1]==1 or take[3]==1)

model.minimize(sum(take*Costs))

if model.solve():
  print("Solution found")
  print("x:", take.value())
else:
  print("No solution found")


Solution found
x: [False  True False False]


**Balanced Academic Curriculum Problem**
The problem is to design a balanced academic curriculum by assigning periods to courses in
a way that the academic load of each period is balanced, i.e., as similar as possible . The
curriculum must obey the following administrative and academic regulations:


*   Academic curriculum: an academic curriculum is defined by a set of courses and a set of prerequisite relationships among them.
*   Number of periods: courses must be assigned within a maximum number of academic periods.
*   Academic load: each course has associated a number of credits or units that
represent the academic effort required to successfully follow it.
*   Prerequisites: some courses can have other courses as prerequisites.
*   Minimum academic load: a minimum number of academic credits per period is
required to consider a student as full time.
*   Maximum academic load: a maximum number of academic credits per period is
allowed in order to avoid overload.
*   Minimum number of courses: a minimum number of courses per period is required to
consider a student as full time.
*   Maximum number of courses: a maximum number of courses per period is allowed in order to avoid overload.

The goal is to assign a period to every course in a way that the minimum and maximum
academic load for each period, the minimum and maximum number of courses for each
period, and the prerequisite relationships are satisfied. An optimal balanced curriculum
minimises the maximum academic load for all periods.

In [32]:
# Input Data
courses = ["C1", "C2", "C3", "C4", "C5"]  # Example courses
credits = [3, 2, 4, 3, 5]  # Corresponding credits
prerequisites = [(0, 2), (1, 3)]  # C1 before C3, C2 before C4
P = 3  # Number of periods
min_load, max_load = 4, 8  # Min and max academic load per period
min_courses, max_courses = 1, 3  # Min and max courses per period

# Decision Variables
X = cp.boolvar(shape=(P, len(courses)))

model=cp.Model()

# Each course exactly once
for c in range(len(courses)):
  model+=cp.sum(X[:,c]) == 1

# Min and max courses per period
for p in range(P):
  model+=(max_courses>=cp.sum(X[p])>=min_courses)

# Min and max academic load per period
for p in range(P):
  model+=(max_courses>=cp.sum(X[p]*credits)>=min_load)

# C1 before C3, C2 before C4 - to improve
model+= X[0][2] < X[0][0] | X[1][2] < X[1][0]
model+= X[0][3] < X[0][1] | X[1][3] < X[1][1]

model.minimize(cp.max(cp.sum(X[p]*credits) for p in range(P)))

if model.solve():
  print("Solution found")
  print("X:", X.value())
else:
  print("No solution found")


Solution found
X: [[False False  True  True False]
 [ True  True False False False]
 [False False False False  True]]


**Traffic lights**
Consider a four way traffic junction with eight traffic lights. Four of the traffic lights are for the
vehicles and can be represented by the variables V1 to V4 with domains {r,g,y} (for red, green and yellow). The other four traffic lights are for the pedestrians and can be
represented by the variables P1 to P4 with domains {r,g}.
Model the problem and find all solutions such that no collisions occur, and the amount of
green lights is maximized.

In [ ]:
car_lights = ["r", "g", "y"]
ped_lights = ["r", "g"]

V = cp.boolvar(shape=(4, 3))  # 4 vehicle lights, 3 states each
P = cp.boolvar(shape=(4, 2))  # 4 pedestrian lights, 2 states each

model = cp.Model()

# Ensure one and only one state is active for each light
for i in range(4):
    model += (sum(V[i]) == 1)
    model += (sum(P[i]) == 1)

for i in range(4):
  model += V[i][1]!=P[i][1] #no green light for pederastian same as for cars
  model += V[i][1]!=P[(i+2) % 4][1] #no green light for pederastian same as opposing for cars
  model += V[i][1]!=V[(i+1) % 4][1] #no green light for light next to each other

model.maximize(sum(V[i][1] for i in range(4)) + sum(P[i][1] for i in range(4))) # Maximize greens

if model.solve():
  print("Solution found")
  print("V:", V.value())
  print("P:", P.value())
else:
  print("No solution found")

Solution found
V: [[ True False False]
 [False  True False]
 [ True False False]
 [False  True False]]
P: [[False  True]
 [ True False]
 [False  True]
 [ True False]]


**Job-shop scheduling** We need to schedule the processing of the tasks from different jobs in several machines.
Each task is labeled by a pair of numbers (m, p) where m is the number of the machine the
task must be processed on and p is the processing time of the task
In the given example, we have three jobs
job 0 = [(0, 3), (1, 2), (2, 2)]
job 1 = [(0, 2), (2, 1), (1, 4)]
job 2 = [(1, 4), (2, 3)]
For example, job 0 has three tasks. The first, (0, 3), must be processed on machine 0 in 3
units of time
There are two types of constraints for the job shop problem
1. Precedence constraint: for any two consecutive tasks in the same job,
the first must be completed before the second can be started.
2. No overlap constraints: a machine can't work on two tasks at the same time

The objective of the job shop problem is to minimize the makespan: the length of time from
the earliest start time of the jobs to the latest end time.

In [102]:
jobs = [[(0, 3), (1, 2), (2, 2)], [(0, 2), (2, 1), (1, 4)], [(0, 0), (1, 2), (2, 3)]]

num_jobs = 3
num_tasks = 3
num_machines = 3
max_times=0
times=[]
for job in jobs:
  for task in job:
    max_times+=task[1]
    times.append(task[1])

# Variables
start_time = cp.intvar(0, max_times, shape = (3,3))
end_times = start_time + cp.cpm_array(times).reshape(3, 3)

model = cp.Model()

# Precedence constraints
for j in range(num_jobs):
  for i in range(len(jobs[j]) - 1):
    model += end_times[j, i] <= start_time[j, i + 1]

# No-overlap constraints
for i in range(num_jobs):
  for m in range(i+1, num_jobs):
    for j in range(num_tasks):
      for k in range(num_tasks):
        if jobs[i][j][0]==jobs[m][k][0]: # Same machine:
          model += ((end_times[i, j] <= start_time[m, k]) | (start_time[i, j] >= end_times[m, k]))

#Objective
model.minimize(cp.max(end_times))

if model.solve():
  print("Solution found")
  print("makespan:", (cp.max(end_times)-cp.min(start_time)).value())
  print("start_time:", start_time.value())
else:
  print("No solution found")

Solution found
makespan: 10
start_time: [[0 3 6]
 [3 5 6]
 [0 0 2]]


**Social golfers**

The coordinator of a local golf club has come to you with the following problem. In their club,
there are 32 social golfers, each of whom play golf once a week, and always in groups of 4.
They would like you to come up with a schedule of play for these golfers, to last as many
weeks as possible, such that no golfer plays in the same group as any other golfer on more
than one occasion. Possible variants of the above problem include: finding a 10-week
schedule with “maximum socialisation”; that is, as few repeated pairs as possible (this has
the same solutions as the original problem if it is possible to have no repeated pairs), and
finding a schedule of minimum length such that each golfer plays with every other golfer at
least once (“full socialisation”). The problem can easily be generalized to that of scheduling
m groups of n golfers over p weeks, such that no golfer plays in the same group as any
other golfer twice (i.e. maximum socialisation is achieved).

In [ ]:
p=10
n=32
groups=cp.intvar(1,32, shape=(10,8,4))

model=cp.Model()

for i in range(10):
  model+=cp.AllDifferent(groups[i])

seen_pairs = set()
for w in range(p):
    for g in range(8):
        for i in range(4):
            for j in range(i + 1, 4):
                golfer_1 = groups[w, g, i]
                golfer_2 = groups[w, g, j]
                pair = (cp.min(golfer_1, golfer_2), cp.max(golfer_1, golfer_2)) #symmetry
                if pair not in seen_pairs:
                    seen_pairs.add(pair)
                else:
                    model += (pair not in seen_pairs)

if model.solve():
  print("Solution found")
  print("groups:", groups.value())
else:
  print("No solution found")

Solution found
groups: [[[ 6  4 32 15]
  [ 9  8  3 17]
  [19 26 24 29]
  [12 10  2  7]
  [31  5 16 22]
  [30 20 21  1]
  [11 18 25 27]
  [14 28 23 13]]

 [[28 14  4 12]
  [19 17  7 26]
  [21  2 27 22]
  [13  3 32 24]
  [31  8 23 10]
  [16 18 30 29]
  [ 9 25  5  6]
  [ 1 20 11 15]]

 [[ 8 25 28 17]
  [22 10 15  9]
  [24 30 21 12]
  [ 2 20  6 27]
  [16 11  7 26]
  [31  5  4  1]
  [23 18  3 14]
  [13 32 29 19]]

 [[24 13 27 10]
  [12 32  7 15]
  [ 9  1  6 18]
  [29 23 30  5]
  [ 8 19 21 25]
  [28 14 22 11]
  [20 17 31  2]
  [16  4 26  3]]

 [[18 17 23 29]
  [24  3 15 16]
  [22 20 12 26]
  [27 10  5 25]
  [28 19 21  1]
  [ 4 11  2  6]
  [14  7  8 31]
  [32 13  9 30]]

 [[13  8 18 15]
  [31 11 20 14]
  [ 7 21  1 10]
  [ 9 24 26 25]
  [32 22 29 30]
  [19 28 23 17]
  [ 2  4 27 16]
  [12  6  3  5]]

 [[28 19 10 13]
  [16 23 24  3]
  [20 30 29  1]
  [12  4  5 11]
  [ 6 32 25  7]
  [ 9 21 26 17]
  [15 18 14 27]
  [22 31  8  2]]

 [[ 1 30 29 32]
  [28 27 26 25]
  [24 22 23 21]
  [20 18 19 16]
  [

**Balanced nurse assignment**

Given a set of patients distributed in a number of hospital zones and an available nursing
staff, one must assign a nurse to each patient in such a way that the work is distributed
evenly between nurses. Each patient is assigned an acuity level corresponding to the
amount of care he requires; the workload of a nurse is defined as the sum of the acuities of
the patients he cares for. A nurse can only work in one zone and there are restrictions both
on the number of patients assigned to a nurse and on the corresponding workload. We
balance the workloads by minimizing their standard deviation.
This problem has two connected aspects: nurses need to be assigned to zones and then
patients of that zone have to be assigned to nurses.

In [144]:
# Data
nurses = ["N1", "N2", "N3", "N4"]
patients = ["P1", "P2", "P3", "P4", "P5", "P6"]
zones = ["A1", "A2", "A3"]
acuity_lev = [2, 3, 4, 5, 2, 1]
max_patients = 2
min_patients = 1
max_workload = 8
min_workload = 1
zone_max = 2

# Decision Variables
x = cp.boolvar((len(nurses), len(patients)))  # x[i][j] = 1 if nurse i handles patient j
y = cp.boolvar((len(nurses), len(zones)))  # y[i][z] = 1 if nurse i is assigned to zone z

# Model
model = cp.Model()

# 1. Each nurse is assigned to exactly one zone
for n in range(len(nurses)):
  model += (cp.sum(y[n, :]) == 1)

# 2. Nurses capacity
for n in range(len(nurses)):
    model += (min_patients <= cp.sum(x[n, :]))  # Enforce lower bound
    model += (cp.sum(x[n, :]) <= max_patients)  # Enforce upper bound

for n in range(len(nurses)):
    model += (min_workload <= cp.sum(x[n, :]*acuity_lev))  # Enforce lower bound
    model += (cp.sum(x[n, :]*acuity_lev) <= max_workload)  # Enforce upper bound

# 3. Each patient is taken care of
for p in range(len(patients)):
  model += (cp.sum(x[:, p]) == 1)

# 4. Zone has its capacity
for z in range(len(zones)):
  model += (cp.sum(y[:, z]) <= zone_max)

# Objective:
mean=sum(acuity_lev) #Instead of mean to not have floting point we multiply be len(nurses), also when substracting
model.minimize(cp.sum((cp.sum(x[n, :]*acuity_lev*len(nurses))-mean)**2 for n in range(len(nurses))))

# Solve the model
if model.solve():
    print("Solution found:")
    print("Nurse assignments to zones and patients:")
    print(x.value())
    print("Patient assignments to zones:")
    print(y.value())
else:
    print("No solution found")


Solution found:
Nurse assignments to zones and patients:
[[False False  True False False False]
 [False  True False False False  True]
 [ True False False False  True False]
 [False False False  True False False]]
Patient assignments to zones:
[[False False  True]
 [False  True False]
 [ True False False]
 [False  True False]]


**Maximal independent sets**
In graph theory, an independent set is a set of vertices in a graph, no two of which are
adjacent. A maximal independent set is an independent set that is not a subset of any other
independent set. A graph may have many maximal independent sets of widely varying sizes:
find a maximal independent set for any given graph. The data provided should use an array
containing, for each node of the graph, the set of adjacent nodes.

In [96]:
edges = [(1,2), (2,3), (2,4)]

X = cp.boolvar(shape=4)

model = cp.Model()

for x1, x2 in edges:
    #model += (X[x1-1] + X[x2-1] <= 1)
    model += (X[x1-1] != 1) | (X[x2-1] != 1)

model.maximize(cp.sum(X))

if model.solve():
  print("Solution found")
  print("X:", X.value())
else:
  print("No solution found")

Solution found
X: [ True False  True  True]


**Resource-Constrained Project Scheduling Problem**


The resource-constrained project scheduling problem involves scheduling a set of jobs, each
with a specific duration and resource requirement, such that the total project duration
(makespan) is minimized. Each job may have precedence constraints, meaning certain jobs
must be completed before others can start. Additionally, there are multiple types of
resources, each with a limited capacity that must not be exceeded at any time.
Given the durations of the jobs, their resource requirements, the capacities of the resources,
and the precedence constraints between the jobs, determine the start times of the jobs that
minimize the makespan while satisfying all constraints.

In [117]:
#to check

# Data
projects = ["P1", "P2", "P3", "P4"]
resources = ["R1", "R2"]
p_resources = [1, 2, 1, 2, 1, 1]  # Resource required by each project
times = [2, 3, 5, 8, 3, 5]        # Duration of each project
capacity = [2,2]           # Resource capacity for each resource

precedence_constraints = [
    (0, 2),  # P1 (index 0) must finish before P3 (index 2)
]

# Variables
starttimes = cp.intvar(0, sum(times), shape=len(p_resources))  # Start time for each project
finishtimes = starttimes + times

# Model
model = cp.Model()

# 1. Precedence Constraints
for p1, p2 in precedence_constraints:
  model += finishtimes[p1]<finishtimes[p2]

# 2. Resource constraints
for r in range(len(resources)):  # Iterate over each resource
    for t in range(sum(times)):  # Iterate over each time point
        # At time `t`, the total usage of resource `r` by all projects must not exceed its capacity
        model += (
            cp.sum(
                [
                    (p_resources[p] == r+1) & (starttimes[p] <= t) & (t < finishtimes[p])
                    for p in range(len(projects))  # Check all projects
                ]
            ) <= capacity[r]  # Enforce resource capacity constraint
        )
# Objective: Minimize the makespan
model.minimize(cp.max(finishtimes))

# Solve the model
if model.solve():
    print("Solution found:")
    print("Start times:", starttimes.value())
    print("Finish times:", finishtimes.value())
    print("Makespan:", max(finishtimes.value()))
else:
    print("No solution found.")

Solution found:
Start times: [4 4 3 0 0 0]
Finish times: [6 7 8 8 3 5]
Makespan: 8


**Disjoint Subsets**
Out of the set of integers 1,...,100 you are given ten different integers (e.g. [81 21 79 4 29 70
28 20 14 7]). From this set A of ten integers you can always find two disjoint non-empty
subsets, S and T, such that the sum of elements in S equals the sum of elements in T. Note:
S union T does not need to be all ten elements of A. Find sets S and T for the given set A.

In [ ]:
A = [81, 81]

S = cp.boolvar(shape=len(A))
T = cp.boolvar(shape=len(A))

model = cp.Model()

model += sum(S*A)==sum(T*A)
model += sum(S)>0
model += sum(S)<len(A)
model += sum(T)>0
model += sum(T)<len(A)
model += S != T

if model.solve():
  print("Solution found")
  print("S:", S.value())
  print("T:", T.value())
else:
  print("No solution found")

Solution found
S: [False  True]
T: [ True False]


**Beer packs**

Given a target number of beers, find the closest combination of 7-packs and 13-packs that
meets or exceeds the target.

In [ ]:
import math
target=70
take=cp.intvar(0, math.floor(target/7), shape=2)
model = cp.Model()
model += take[0] *7 + take[1] *13>=target
#model.minimize(sum(take))
model.minimize(take[0] *7 + take[1] *13)
if model.solve():
  print("Solution found")
  print("x:", take.value())
else:
  print("No solution found")

Solution found
x: [10  0]


**Agatha murder**
Someone in Dreadsbury Mansion killed Aunt Agatha. Agatha, the butler, and Charles live in
Dreadsbury Mansion, and are the only ones to live there. A killer always hates, and is
no richer than his victim. Charles hates noone that Agatha hates. Agatha hates everybody
except the butler. The butler hates everyone not richer than Aunt Agatha. The butler hates
everyone whom Agatha hates. Noone hates everyone. Who killed Agatha?

In [147]:
# Decision Variables for each suspect representing if they are guilty
# Define decision variables for "hates" and "richer"
hates = cp.boolvar(shape=(3, 3))
richer = cp.boolvar(shape=(3, 3))

# Create the model
model = cp.Model()

# Add the constraints based on the problem statement
model += hates[0, 0] == 1  # Agatha hates herself
model += hates[0, 1] == 0  # Agatha does not hate the butler
model += hates[0, 2] == 1  # Agatha hates Charles

model += hates[2, 0] != 1  # Charles does not hate Agatha
model += hates[2, 2] != 1  # Charles does not hate himself

model += hates[1, 0] == 1  # The butler hates Agatha
model += hates[1, 1] == 0  # The butler does not hate himself
model += hates[1, 2] == 1  # The butler hates Charles

model += richer[0, 0] == 0  # Agatha is not richer than herself
model += richer[0, 2] == 0  # Agatha is not richer than Charles

# Solve the model
model.solve()

# Extract results
hates_result = hates.value()
richer_result = richer.value()

print("Hates Matrix:\n", hates_result)
print("Richer Matrix:\n", richer_result)

Hates Matrix:
 [[True False True]
 [True False True]
 [False None False]]
Richer Matrix:
 [[False None False]
 [None None None]
 [None None None]]


In [151]:
# Variables: 1 if a person killed Agatha, 0 otherwise
agatha_killed = cp.boolvar(name="agatha_killed")
butler_killed = cp.boolvar(name="butler_killed")
charles_killed = cp.boolvar(name="charles_killed")

# Variables for who hates whom: 1 if hates, 0 otherwise
agatha_hates_agatha = cp.boolvar(name="agatha_hates_agata")
agatha_hates_butler = cp.boolvar(name="agatha_hates_butler")
agatha_hates_charles = cp.boolvar(name="agatha_hates_charles")
butler_hates_butler = cp.boolvar(name="butler_hates_butler")
butler_hates_agatha = cp.boolvar(name="butler_hates_agatha")
butler_hates_charles = cp.boolvar(name="butler_hates_charles")
charles_hates_charles = cp.boolvar(name="charles_hates_charles")
charles_hates_agatha = cp.boolvar(name="charles_hates_agatha")
charles_hates_butler = cp.boolvar(name="charles_hates_butler")

# Variables for wealth comparison: 1 if richer, 0 otherwise
butler_richer_than_agatha = cp.boolvar(name="butler_richer_than_agatha")
charles_richer_than_agatha = cp.boolvar(name="charles_richer_than_agatha")

# Create model
model = cp.Model()

# 1. Someone in the mansion killed Agatha
model += (agatha_killed + butler_killed + charles_killed == 1)

# 2. A killer always hates and is no richer than their victim
model += (agatha_hates_agatha).implies(agatha_killed)
model += (butler_hates_agatha & ~butler_richer_than_agatha).implies(butler_killed)
model += (charles_hates_agatha & ~charles_richer_than_agatha).implies(charles_killed)

# 3. Charles hates no one Agatha hates
model += agatha_hates_agatha.implies(~butler_hates_agatha)
model += agatha_hates_butler.implies(~butler_hates_butler)
model += agatha_hates_charles.implies(~butler_hates_charles)

# 4. Agatha hates everybody except the butler
model += (agatha_hates_agatha == 1)
model += (agatha_hates_butler == 0)
model += (agatha_hates_charles == 1)

# 5. The butler hates everyone not richer than Aunt Agatha
model += (butler_hates_agatha == 0)
model += ~butler_richer_than_agatha.implies(butler_hates_butler)
model += ~charles_richer_than_agatha.implies(butler_hates_charles)

# 6. The butler hates everyone Agatha hates
model += agatha_hates_agatha.implies(charles_hates_agatha)
model += agatha_hates_butler.implies(charles_hates_butler)
model += agatha_hates_charles.implies(charles_hates_charles)

# 7. No one hates everyone
model += (agatha_hates_butler + agatha_hates_charles + agatha_hates_agatha< 3)
model += (butler_hates_agatha + butler_hates_charles + butler_hates_butler< 3)
model += (charles_hates_agatha + charles_hates_butler + charles_hates_charles< 3)


if model.solve():
  print("Solution found")
  print("agatha_killed:", agatha_killed.value())
  print("butler_killed:", butler_killed.value())
  print("charles_killed:", charles_killed.value())
else:
  print("No solution found")

Solution found
agatha_killed: True
butler_killed: False
charles_killed: False


**Langford's number problem.**
Arrange 2 sets of positive integers 1..k to a sequence, such that, following the first
occurence of an integer i, each subsequent occurrence of i, appears i+1 indices later than
the last. For example, for k=4, a solution would be 41312432.

In [71]:
k=4
n=2*k

numbers = cp.intvar(1, k, shape=8)

model=cp.Model()

#Every number 2 times at maximum
for i in range(1,k+1):
  model += cp.Count(numbers, i) == 2

# Constraint: Langford sequence rules
for num in range(1, k + 1):
    # Positions for the two occurrences of 'num'
    pos1 = cp.intvar(0, n - 1)
    pos2 = cp.intvar(0, n)
    model += (pos2 - pos1 == num + 1)  # Gap constraint
    model += (pos1 < pos2)  # Ensure order
    model += (numbers[pos1] == num)  # First occurrence
    model += (numbers[pos2] == num)  # Second occurrence

if model.solve():
  print("Solution found")
  print("numbers:", numbers.value())
else:
  print("No solution found")

Solution found
numbers: [2 3 4 2 1 3 1 4]


For the well-known graph coloring problem you need to assign colors to the nodes in a graph, such
that no two adjacent nodes (connected by an edge) have the same color.
1. Complete the following model to find the minimal number of colors for coloring the graph.

Note that the values of the parameters correspond to a graph with 4 nodes and the 3 edges (an
edge between node 1 and 2, another between 2 and 3 and another between node 2 and 4).



In [ ]:
num_nodes = 4
edges = [(1,2), (2,3), (2,4)]
max_colors = num_nodes
nodecolors = cp.intvar(1, max_colors, shape=num_nodes)

model=cp.Model()

for nodes in edges:
  model += nodecolors[nodes[0]-1]!=nodecolors[nodes[1]-1]

model.minimize(sum(nodecolors))

if model.solve():
  print("Solution found")
  print("nodecolors:", nodecolors.value())
else:
  print("No solution found")

Solution found
nodecolors: [1 2 1 1]


Consider a variant of this problem. Given the nodes and a minimal coloring, find the edges of
a graph such that this is a valid coloring of that graph. Also consider that self-loops and duplicate
edges are not allowed.

In [97]:
num_nodes = 4
num_edges = 3
max_colors = num_nodes
nodecolors = [1,2,1,2]
edges = cp.intvar(1, num_nodes, shape=(num_edges, 2))

nodecolors = cp.cpm_array(nodecolors)

model=cp.Model()

for nodes in edges:
  model += nodecolors[nodes[0]-1]!=nodecolors[nodes[1]-1]

#not self edge
for nodes in edges:
  model += nodes[0]!=nodes[1]

#no duplicate
for i in range(num_edges):
    for j in range(i + 1, num_edges):
        # Ensure edges are not the same in both orientations
        model += (edges[i, 0] != edges[j, 0]) | (edges[i, 1] != edges[j, 1])
        model += (edges[i, 0] != edges[j, 1]) | (edges[i, 1] != edges[j, 0])

if model.solve():
  print("Solution found")
  print("edges:", edges.value())
else:
  print("No solution found")

Solution found
edges: [[1 4]
 [3 2]
 [1 2]]
